In [2]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [90]:
N_EPOCHS = 200
N_TRIALS = 3
BATCH_SIZE = 256

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [98]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
x, y = utils.load_data("../data/cifar-10-batches-py/data_batch_1")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

train_size = x_train.shape[0]

In [99]:
x_train = np.clip(x_train + np.random.normal(0, 20, x_train.shape), 0, 255)
x_val = np.clip(x_val + np.random.normal(0, 20, x_val.shape), 0, 255)
x_test = np.clip(x_test + np.random.normal(0, 20, x_test.shape), 0, 255)

In [100]:
assessment_model = ClassifierModel(output_shape=10, **models_hyperparameters["assessment_model"])

assessment_model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
        
assessment_model.fit(x_train, y_train, epochs=60, batch_size=BATCH_SIZE)

Epoch 1/60
28/28 [==============================] - 1s 27ms/step - loss: 2.5078 - accuracy: 0.1290
Epoch 2/60
28/28 [==============================] - 1s 28ms/step - loss: 2.2322 - accuracy: 0.1889
Epoch 3/60
28/28 [==============================] - 1s 27ms/step - loss: 2.1179 - accuracy: 0.2233
Epoch 4/60
28/28 [==============================] - 1s 28ms/step - loss: 2.0264 - accuracy: 0.2541
Epoch 5/60
28/28 [==============================] - 1s 28ms/step - loss: 1.9485 - accuracy: 0.2770
Epoch 6/60
28/28 [==============================] - 1s 28ms/step - loss: 1.8782 - accuracy: 0.3066
Epoch 7/60
28/28 [==============================] - 1s 28ms/step - loss: 1.8393 - accuracy: 0.3201
Epoch 8/60
28/28 [==============================] - 1s 27ms/step - loss: 1.7941 - accuracy: 0.3416
Epoch 9/60
28/28 [==============================] - 1s 29ms/step - loss: 1.7620 - accuracy: 0.3513
Epoch 10/60
28/28 [==============================] - 1s 28ms/step - loss: 1.7259 - accuracy: 0.3673
Epoch 11/

In [101]:
y_pred_assessment = assessment_model.predict(x_train, verbose=0)

losses_assessment = utils.calculate_loss_per_sample(y_train, y_pred_assessment, loss)
order_assessment = np.argsort(losses_assessment)

In [102]:
results = {}

# test_models = ["test_model_1", "test_model_2", "test_model_3"]
test_models = ["test_model_1"]

In [103]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
for test_model in test_models:
    print(test_model)
    model_scores = []

    for _ in tqdm(range(N_TRIALS)):
        model = ClassifierModel(output_shape=10, **models_hyperparameters[test_model])
        
        model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
        
        for i in range(N_EPOCHS):
            n_samples = int(np.tanh(2 * (i + 1)/ N_EPOCHS) * train_size)
            samples_ids = order_assessment[:n_samples]
            
            model.fit(
                x_train[samples_ids], y_train[samples_ids], 
                validation_data=(x_val, y_val),
                epochs=1, batch_size=BATCH_SIZE,
                verbose=1, callbacks=[early_stopping]
            )
        
        _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=0)
        
        model_scores.append(accuracy)
    
    results[test_model] = model_scores

test_model_1


  0%|          | 0/3 [00:00<?, ?it/s]

 7/22 [========>.....................] - ETA: 0s - loss: 0.8572 - accuracy: 0.6914

In [ ]:
results